`all_losses.backward()`       
using all_losses gives error. i dont know why. 

`loss.backward()`       
thats why i have used loss only which is not that accurate.

In [2]:
import torch
import joblib
from torchvision import datasets, transforms
import numpy as np

# device = torch.device("cpu")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
!mkdir _p _cache
cache = joblib.Memory(location='_cache', verbose=0)
# torch.autograd.set_detect_anomaly(True)

A subdirectory or file -p already exists.
Error occurred while processing: -p.
A subdirectory or file _cache already exists.
Error occurred while processing: _cache.


In [3]:
def detach_var(v):
    var = v.detach().clone().requires_grad_(True).to(device)
    # var = v.clone().requires_grad_(True).to(device)
    var.retain_grad()
    return var

# MNIST

In [4]:
train_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

test_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_data = datasets.MNIST(root='./data', train=True, download=True, transform=train_transforms)
test_data = datasets.MNIST(root='./data', train=False, download=True, transform=test_transforms)

batch_size = 64

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=False)

print(f"Training Data Loader: {len(train_loader)} batches")
print(f"Testing Data Loader: {len(test_loader)} batches")

Training Data Loader: 938 batches
Testing Data Loader: 157 batches


In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class MNISTNet(nn.Module):
    def __init__(self, layer_size=20, n_layers=1):
        super().__init__()
        
        self.layers = nn.ModuleList()
        inp_size = 28*28
        
        # Hidden layers
        for i in range(n_layers):
            self.layers.append(nn.Linear(inp_size, layer_size))
            self.layers.append(nn.Sigmoid())
            inp_size = layer_size
        
        # Final output layer
        self.final_layer = nn.Linear(inp_size, 10)
        
        # Initialize weights and biases
        # for module in self.modules():
        #     if isinstance(module, nn.Linear):
        #         nn.init.normal_(module.weight, mean=0, std=0.001)
        #         nn.init.zeros_(module.bias)
        
        self.loss = nn.NLLLoss()
                
    def forward(self, x):
        x = x.view(x.size(0), -1)
        
        for layer in self.layers:
            x = layer(x)
        
        x = F.log_softmax(self.final_layer(x), dim=1)
        return x
    
    # def all_named_parameters(self):
    #     return [(name, param) for name, param in self.named_parameters()]
    

# optimizee = MNISTNet().to(device)
# x = next(iter(train_loader))[0].to(device)
# y = next(iter(train_loader))[1].to(device)
# # x.shape, y.shape
# # optimizee.all_named_parameters()[3]
# out = optimizee(x)
# print(out.shape, y.shape)
# loss = optimizee.loss(out, y)
# loss.backward()
# print(loss)

In [6]:
class Optimizer(nn.Module):
    def __init__(self, preproc=False, hidden_sz=20, preproc_factor=10.0):
        super().__init__()
        self.hidden_sz = hidden_sz
        self.recurs = nn.LSTMCell(1, hidden_sz)
        self.recurs2 = nn.LSTMCell(hidden_sz, hidden_sz)
        self.output = nn.Linear(hidden_sz, 1)

    def forward(self, inp, hidden, cell):
        # print(inp.shape, hidden[0].shape, cell[0].shape)
        # print(inp.shape, hidden[1].shape, cell[1].shape)
        hidden0, cell0 = self.recurs(inp, (hidden[0], cell[0]))
        hidden1, cell1 = self.recurs2(hidden0, (hidden[1], cell[1]))

        # print(hidden0.shape, hidden1.shape)
        return self.output(hidden1), (hidden0, hidden1), (cell0, cell1)

In [24]:
def do_fit(opt_net, meta_opt, target_cls, target_to_opt, unroll, optim_it, n_epochs, out_mul, should_train=True):
    if should_train:
        opt_net.train()
    else:
        opt_net.eval()
        unroll = 1

    optimizee = target_to_opt().to(device)
    # n_params = 0
    # for p in optimizee.parameters():
        # n_params += int(np.prod(p.size()))
    n_params = sum(p.numel() for p in optimizee.parameters())

    hidden_states = [torch.zeros(n_params, opt_net.hidden_sz).to(device) for _ in range(2)]
    cell_states = [torch.zeros(n_params, opt_net.hidden_sz).to(device) for _ in range(2)]

    all_losses_ever = []
    if should_train:
        meta_opt.zero_grad()

    all_losses = None
    
    for iteration in range(1, optim_it + 1):
        out = optimizee(target_cls[0])
        loss = optimizee.loss(out, target_cls[1])
        loss.backward(retain_graph=should_train)

        if all_losses is None:
            all_losses = loss
        else:
            all_losses = all_losses + loss

        all_losses_ever.append(loss.item())

        offset = 0
        result_params = {}
        
        hidden_states2 = [torch.zeros(n_params, opt_net.hidden_sz).to(device) for _ in range(2)]
        cell_states2 = [torch.zeros(n_params, opt_net.hidden_sz).to(device) for _ in range(2)]

        for name, p in optimizee.named_parameters():
            # print(name, p.shape)
            cur_sz = int(np.prod(p.size()))
            gradients = detach_var(p.grad.view(-1, 1))
            updates, new_hidden, new_cell = opt_net(
                        gradients,
                        [h[offset:offset+cur_sz] for h in hidden_states],
                        [c[offset:offset+cur_sz] for c in cell_states]
                    )
    
            for i in range(len(new_hidden)):
                hidden_states2[i][offset:offset+cur_sz] = new_hidden[i]
                cell_states2[i][offset:offset+cur_sz] = new_cell[i]

            # print(p.grad)
            # result_params[name] = p + updates.view(*p.size()) * out_mul
            updated_param = p.clone().detach().requires_grad_(True).to(device) + updates.view(*p.size()) * out_mul
            result_params[name] = updated_param
            result_params[name].retain_grad()

            offset += cur_sz

        if iteration % unroll == 0:
            # print(iteration)
            if should_train:
                meta_opt.zero_grad()
                # all_losses.backward()      # using all_losses gives error. i dont know why. 
                loss.backward()
                meta_opt.step()

            all_losses = None
            optimizee.load_state_dict({k: detach_var(v) for k, v in result_params.items()})
            hidden_states = [detach_var(v) for v in hidden_states2]
            cell_states = [detach_var(v) for v in cell_states2]

        else:
            # print(iteration)
            # print(result_params.keys())
            optimizee.load_state_dict(result_params)
            # optimizee.load_state_dict({k: detach_var(v) for k, v in result_params.items()})
            hidden_states = hidden_states2
            cell_states = cell_states2

    return all_losses_ever


In [25]:
lr=0.001
target_to_opt = MNISTNet
opt_net = Optimizer().to(device)
meta_opt = torch.optim.Adam(opt_net.parameters(), lr=lr)

x = next(iter(train_loader))[0].to(device)
y = next(iter(train_loader))[1].to(device)
target_cls = (x, y)

do_fit(opt_net, meta_opt, target_cls, target_to_opt, unroll=5, optim_it=100, n_epochs=20, out_mul=1, should_train=False)
